## BU_L1

In [2]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
import warnings
warnings.simplefilter("ignore")
from Invtsim import *
import numpy as np
import pandas as pd
from scipy.stats import norm
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace
from tqdm import tqdm

gap1    = 365
gap2    = 1548
train   = pd.read_pickle(f"{path1}721past_1913.pkl").reset_index(drop=True)
test    = pd.read_pickle(f"{path1}721future_28.pkl").reset_index(drop=True)
S       = pd.read_pickle(f"{path1}721S_df.pkl")
tags    = pd.read_pickle(f"{path1}tags.bin")

ts_id   = test[['unique_id', 'ds']]
tr_id   = train[['unique_id', 'ds']]
trts_id = tr_id[tr_id['ds']> gap2]
trtr_id = tr_id[tr_id['ds']<= gap2]
'''
lgbs = pd.read_pickle("lgbInvtSim_L1.pkl")
etss = pd.read_pickle("etsInvtSim_L1.pkl")
bus = pd.concat([lgbs,etss])'''
del train, test, trtr_id

In [3]:
trts_id['unique_id'].unique()

array(['TOTAL/FOODS/FOODS_1/FOODS_1_001',
       'TOTAL/FOODS/FOODS_1/FOODS_1_002',
       'TOTAL/FOODS/FOODS_1/FOODS_1_003', ...,
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516'],
      dtype=object)

## Order BU

### L1

In [ ]:
################################### BU
lt = 1
on   = ['ot_90','ot_95','ot_99']
names = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint','ets_base', 'ets_bu', 'ets_td', 'ets_mint']
bottomup = [BottomUp()]
bu = HierarchicalReconciliation(reconcilers=bottomup)
eobu,lobu = [], []

lgbs = pd.read_pickle(f"721lgbInvtSim_L{lt}.pkl")
etss = pd.read_pickle(f"721etsInvtSim_L{lt}.pkl")
bus = pd.concat([lgbs,etss])

for i in tqdm(names):
    obu = pd.concat([ts_id, bus[bus['name'] == i][on]],axis=1)
    lobu.append(bu.reconcile(Y_hat_df=obu, S=S, tags=tags).iloc[:,-3:])
orderbu = pd.concat([bus['name'], pd.concat(lobu)],axis=1)
#ets_orderbu.to_pickle("Order_ets_BU.pkl")
orderbu.to_pickle(f"721OrderBU_L{lt}.pkl")

 50%|███████████████████████████████████████████████████████████████████████████                                                                           | 4/8 [05:05<05:03, 75.83s/it]

In [ ]:
orderbu

In [ ]:
lt = 1
on   = ['ot_90','ot_95','ot_99']
#pd.read_pickle(f"721OrderBU_L{lt}.pkl")
pd.read_pickle(f"721BUOrder_L{lt}.pkl")[on]

In [ ]:
pd.read_pickle(f"BUOrder_L{lt}.pkl")[on]

## Fixed Order Simulation:

In [ ]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
from Invtsim import *
import numpy as np
import pandas as pd
from scipy.stats import norm
import warnings
warnings.simplefilter("ignore")
from tqdm import tqdm
def loop2( truth, NAME:str, fcst, residual = 0,
         gap1: int = 28, gap2:int = 1913,
         n = None, 
         df_list:list = [], 
         L_ = 3):
    if n == None:
        n = int(len(truth)/gap1)
    for i in tqdm(range(n)):
        fcst_1  = fcst[i*gap1:(i+1)*gap1]
        truth_1 = truth[i*gap1:(i+1)*gap1]
        #res_1   = residual[i*gap2:(i+1)*gap2]
        df = InvtSim(fcst = 0, truth = truth_1, residual = residual, name = NAME, L=L_)##################
        res = df.ob_all_t_fixedcase(fixed_order = fcst_1)
        df_list.append(res.reset_index(drop = True))
    print("-----------DOWN!------------")
    return pd.concat(df_list).reset_index(drop=True)

## BU-FIXED SIMULATION:

### L1

In [ ]:
lead_time = 1
BU   = pd.read_pickle(f"721OrderBU_L{lead_time}.pkl")
truth= pd.read_pickle(f"{path1}721future_28.pkl")['y']
Invt_df = []
#names   = [ "lgb_base", "lgb_bu", "ets_base", "ets_bu"]
names = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint','ets_base', 'ets_bu', 'ets_td', 'ets_mint']
for i in names:
    df = loop2(fcst = BU[BU['name'] == i].iloc[:,-3:], truth = truth, residual = [], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"721BUOrder_L{lead_time}.pkl")

In [ ]:
pd.read_pickle(f"721BUOrder_L{lead_time}.pkl")

In [5]:
9561664/8

1195208.0

In [7]:
1195208/42686

28.0